### Abstract:
In this notebook we're going to dock PRD_002214 over 6LU7 (COVID-19 main protease https://pdb101.rcsb.org/motm/242). The objective is evaluating the effectiveness for pharmacological usage.

### References:
https://kodomo.fbb.msu.ru/~golovin/oddt.html  
https://github.com/vegetablejuiceftw/coronavirus-begone/blob/master/coronavirus_begone.ipynb  
https://openbabel.org/docs/dev/Command-line_tools/babel.html  
http://openbabel.org/docs/current/UseTheLibrary/PythonDoc.html#examples
https://oddt.readthedocs.io/en/latest/_modules/oddt/docking/AutodockVina.html  
http://python.zirael.org/e-openbabel10.html

In [758]:
from rdkit import Chem
from rdkit.Chem import Draw
import oddt
from oddt.docking import autodock_vina
import nglview
import uuid
import ipywidgets as widgets
from IPython.display import HTML, display
import tabulate

Let's define some utility functions

In [720]:
class oddt_trajectory(nglview.Structure, nglview.Trajectory):
    ext = "pdb"  # or gro, cif, mol2, sdf
    params = {}
    def __init__(self,res):
        self.m = res[0]
        self.res = res
        self.id = str(uuid.uuid4())
    def get_structure_string(self):
        return self.m.write('pdb')
    def get_coordinates( self, index ):
        return self.res[index].coords
    @property
    def n_frames( self ):
        return len(self.res)

class ob_structure(nglview.Structure):
    ext = 'pdb'
    params = {}
    def __init__(self,m):
        self.m = m
        self.id = str(uuid.uuid4())
    def get_structure_string(self):
        return self.m.write('pdb')

nglview.ODDTTrajectory = oddt_trajectory
nglview.OBStructure = ob_structure

We open the models we previously prepared. Please refer to the other notebook for more details

In [727]:
_6LU7 = next(oddt.toolkits.ob.readfile('pdbqt','datasets/6LU7.pdbqt'))
PRD_002214 = next(oddt.toolkits.ob.readfile('pdbqt','datasets/PRD_002214.pdbqt'))

Let's show the molecule and it's ligand inhibitor

In [728]:
w1 = nglview.NGLWidget()
w1.add_structure(nglview.OBStructure(_6LU7))
w1.add_representation('ball+stick', selection='all')
w1._remote_call("setSize", target="Widget", args=["", ""])
w1.center()
w1.parameters=dict(clipDist=-200)
w1.layout.flex = '1 1 0'

w2 = nglview.NGLWidget()
w2.add_structure(nglview.OBStructure(PRD_002214))
w2.add_representation('ball+stick', selection='all')
w2._remote_call("setSize", target="Widget", args=["", ""])
w2.center()
w2.parameters=dict(clipDist=-200)
w2.layout.flex = '1 1 0'

box = widgets.HBox(children=(w1,w2))
display(box)

We're ready to dock the ligand on the right to the [bad guy](https://pdb101.rcsb.org/motm/242) on the left.  
Let's instantiate the main object

In [725]:
# if you use auto_ligand the center patameter is not used, since the box center is determined as a centroid of ligand's heavy atoms.
# vina --receptor cov_receptor.pdbqt --ligand cov_inhibitor.pdbqt --out cov_out.pdbqt --log ligand.log --exhaustiveness 2 --center_x -10 --center_y 15 --center_z 70 --size_x 10 --size_y 20 --size_z 20
engine = autodock_vina(protein='datasets/6LU7.pdbqt',exhaustiveness=2,center=(-10,15,70),size=(10,20,20),n_cpu=2,autocleanup=False)
engine.tmp_dir = 'datasets'

This is the actual part where we do the docking (it will take a while...)

In [729]:
dock_res = engine.dock(ligands=PRD_002214)

Now we can finally show the result of the docking.  
You can hit the play button to show the different docking proposals we got from Autodock Vina

In [1]:
pres = [{
    "type": "ball+stick",
    "params": {
        "sele": "all"
    }
}]

structure = nglview.OBStructure(_6LU7)
trajectory = nglview.ODDTTrajectory(dock_res)
view = nglview.NGLWidget()
view.add_structure(structure)
view.add_trajectory(trajectory)
view.representations = pres
view

NameError: name 'nglview' is not defined

Let's print the results.

In [782]:
table_res = []
vina_keys = list(filter(lambda elem: elem.startswith("vina_"), dock_res[0].data.keys()))
table_res.append(vina_keys)

for res in dock_res:
    table_res_record = list({ your_key: res.data[your_key] for your_key in vina_keys }.values())
    table_res.append(table_res_record)

display(HTML(tabulate.tabulate(table_res, tablefmt='html')))

vina_affinity,vina_rmsd_lb,vina_rmsd_ub,vina_rmsd_input,vina_rmsd_input_min
-5.1,0.000,0.000,69.184746,69.18368
-3.6,2.437,5.570,70.61464,70.61387
-3.5,2.299,4.381,69.83454,69.83401
-3.2,2.401,5.262,69.176445,69.17606


We're interested in the Affinity parameter.  
Affinity is the term used in the Autodock Vina output to indicate the predicted binding energy (lower values indicating tighter binding).